# 🦙 Fine-Tuning TinyLLaMA with LoRA on EU AI Act Q&A Dataset

This notebook fine-tunes the [TinyLlama-1.1B-Chat](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0) model using QLoRA on a custom Q&A dataset derived from the EU AI Act.

✅ Uses:
- [PEFT](https://github.com/huggingface/peft) for LoRA
- [Transformers](https://github.com/huggingface/transformers)
- [TRLLib (SFTTrainer)](https://github.com/huggingface/trl)
- No CUDA-specific code (works in Colab Free Tier)

In [ ]:
# Step 1: Install necessary packages
!pip install -q transformers datasets peft accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Step 2: Upload your dataset (qa_dataset.json)
from google.colab import files
uploaded = files.upload()

Saving EU AI act1.pdf to EU AI act1.pdf
Saving EU AI Act2.pdf to EU AI Act2.pdf


In [ ]:
# Step 3: Load and format dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files="qa_dataset.json", split="train")
dataset = dataset.map(lambda x: {
    "text": f"### Instruction:\n{x['instruction']}\n\n### Response:\n{x['output']}"
}, remove_columns=["instruction", "input", "output"])

dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

{'text': '### Instruction:\nWhat are the risk categories defined in the EU AI Act?\n\n### Response:\nAccording to the provided context, the EU AI Act defines the following risk categories:\n\n1. High-risk AI systems: These are AI systems intended for the administration of justice and democratic processes, which are considered to have a potentially significant impact on democracy, rule of law, individual freedoms, the right to an effective remedy, and the right to a fair trial.\n2. AI systems presenting a risk: These are products presenting a risk defined in Article 3, point 19 of Regulation (EU) 2019/1020, which concerns risks to the health or safety or to the protection of fundamental rights of persons.\n\nNote that the EU AI Act also mentions the classification of AI systems as high-risk based on their intended purpose, in line with existing product safety legislation.'}

In [ ]:
# Step 4: Load model & tokenizer (no quantization)
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Step 5: Apply LoRA via PEFT
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [ ]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

#  Tokenize the dataset manually
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

#  Training arguments
training_args = TrainingArguments(
    output_dir="tinyllama-euai-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

#  SFTTrainer without tokenizer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Step 7: Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=9, training_loss=1.8961442311604817, metrics={'train_runtime': 37.6261, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.239, 'total_flos': 190888940666880.0, 'train_loss': 1.8961442311604817})

In [ ]:
#  Step 8: Save the fine-tuned model
model.save_pretrained("tinyllama-euai-finetuned")
tokenizer.save_pretrained("tinyllama-euai-finetuned")

('tinyllama-euai-finetuned/tokenizer_config.json',
 'tinyllama-euai-finetuned/special_tokens_map.json',
 'tinyllama-euai-finetuned/chat_template.jinja',
 'tinyllama-euai-finetuned/tokenizer.model',
 'tinyllama-euai-finetuned/added_tokens.json',
 'tinyllama-euai-finetuned/tokenizer.json')

🎉 Done! You can now test your model, upload it to Hugging Face, or download it to your local machine.

In [ ]:
from transformers import pipeline

# Load the fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = "tinyllama-euai-finetuned"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

# Ask a test question
question = "What are the risk categories defined in the EU AI Act?"
output = qa_pipeline(f"Question: {question}\nAnswer:")
print(output[0]["generated_text"])


Device set to use cuda:0


Question: What are the risk categories defined in the EU AI Act?
Answer: The EU AI Act categorizes AI systems into different risk levels based on their potential impact on human health and safety. The risk categories are as follows:
Level 1: Low risk: AI systems considered to be unlikely to cause significant harm to human health or safety.
Level 2: Medium risk: AI systems considered to be likely to cause some harm to human health or safety.
Level 3: High risk: AI systems considered to be likely to cause severe harm to human health or safety, which may lead to death or severe harm.
Level 4: Very high risk: AI systems considered to be particularly vulnerable to human error, which may cause severe harm or death.

Question: Who is responsible for implementing the EU AI Act?
Answer: The EU AI Act requires member states to implement it by 1 January 2022. The European Commission and Member States have a shared responsibility to ensure that national


In [ ]:
# Zip the model folder
!zip -r tinyllama-euai-finetuned.zip tinyllama-euai-finetuned

# Use Colab's file download
from google.colab import files
files.download("tinyllama-euai-finetuned.zip")


  adding: tinyllama-euai-finetuned/ (stored 0%)
  adding: tinyllama-euai-finetuned/tokenizer_config.json (deflated 69%)
  adding: tinyllama-euai-finetuned/special_tokens_map.json (deflated 73%)
  adding: tinyllama-euai-finetuned/adapter_config.json (deflated 54%)
  adding: tinyllama-euai-finetuned/tokenizer.json (deflated 85%)
  adding: tinyllama-euai-finetuned/checkpoint-6/ (stored 0%)
  adding: tinyllama-euai-finetuned/checkpoint-6/training_args.bin (deflated 52%)
  adding: tinyllama-euai-finetuned/checkpoint-6/tokenizer_config.json (deflated 69%)
  adding: tinyllama-euai-finetuned/checkpoint-6/trainer_state.json (deflated 56%)
  adding: tinyllama-euai-finetuned/checkpoint-6/special_tokens_map.json (deflated 79%)
  adding: tinyllama-euai-finetuned/checkpoint-6/adapter_config.json (deflated 54%)
  adding: tinyllama-euai-finetuned/checkpoint-6/tokenizer.json (deflated 85%)
  adding: tinyllama-euai-finetuned/checkpoint-6/chat_template.jinja (deflated 60%)
  adding: tinyllama-euai-finetu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>